### Домашнее задание «Продвинутый pandas 2»

#### Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [55]:
import pandas as pd

In [56]:
ratings = pd.read_csv('C:/ds22/1_pyda/13_pd_adv2/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [57]:
result = ratings.groupby('userId').agg({'movieId':'count', 'timestamp':['min','max']})
result.head()

movieId   timestamp            
         count         min         max
userId                                
1           20  1260759108  1260759205
2           76   835355395   835356246
3           51  1298861589  1298932787
4          204   949778714   949982274
5          100  1163373044  1163375145

In [58]:
ratings['userId'].nunique() == len(result)
ratings['userId'].nunique()

671

In [60]:
result = result[result['movieId']['count'] > 100]
result['userlivetime'] = result.apply(lambda row: row['timestamp']['max']-row['timestamp']['min'], axis=1)
result.sort_values(by='userlivetime')
# После данной сортировки привлекают внимание строки с очень маленьким значенем времени жиэни (200сек, 800 сек) для такого 
# количества выставленных оценок (109, 140). В реальной ситуации это может быть направлением для дальнейшего анализа.

movieId   timestamp             userlivetime
         count         min         max             
userId                                             
536        109   829471719   829471925          206
609        140  1029869519  1029870367          848
316        140  1460822646  1460823562          916
217        104  1108160000  1108160981          981
110        120   840100360   840101508         1148
...        ...         ...         ...          ...
624       1735  1019123866  1476616373    457492507
15        1700   997937239  1469330735    471393496
427        294   939332587  1415901843    476569256
547       2391   974777109  1476587644    501810535
380       1063   949367125  1465156786    515789661

[258 rows x 4 columns]

In [61]:
user_lt = result.userlivetime.mean()/len(result)

In [62]:
print(f'Cреднее время жизни пользователей, которые выставили более 100 оценок - {user_lt/3600:.4} часа.')

Cреднее время жизни пользователей, которые выставили более 100 оценок - 43.15 часа.


#### Задание 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов  

Необходимо сформировать две таблицы:  
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента  
- аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


In [14]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [15]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [16]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [17]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [18]:
#1. Cформируем таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
revenue = rzd.merge(auto, how = 'outer')
revenue

,client_id,rzd_revenue,auto_revenue
0,111,1093.0,NaN
1,112,2810.0,NaN
2,113,10283.0,57483.0
3,114,5774.0,83.0
4,115,981.0,912.0
5,116,NaN,4834.0
6,117,NaN,98.0


In [19]:
revenue = revenue.merge(air, how = 'outer')
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [20]:
#2. Cформируем таблицу таблицу по типам выручки с указанием адреса клиента
client_data = client_base.merge(revenue, how = 'left')
client_data

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


#### Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


Ответ:  
Если бы мне были доступны данные о координатах пользователей, я бы постаралась связать из 
с моментами посещения сайтов или совершения покупок. В целом эту информацию можно использовать 
для показа персонализированных предложений от компаний, которые находятся недалеко от пользователя. 
Именно это делают службы доставки. Например, предложить заказать еду из ресторана неподалеку, 
гарантировав быструю доставку.
Удаленность координат одного пользователя может быть связана с тем, что находясь в офисе и дома, пользователь может покупать различные товары. Например, в офисе постоянно заказывает канцтовары, а дома продукты и детские товары. Соответственно интересы пользователя можно разделить в зависимости от его локации.
Дополнительным признаком, который можно получить из координат, является адрес. Есть технология обратного геокодирования (reverse geocoding), которая позволяет это сделать. 